In [1]:
import pandas as pd
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

d:\Python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
Using TensorFlow backend.


# Подготовка Датасета

In [2]:

### Prepare data
lyrics = pd.read_csv('dataHPMRM.csv')
# lyrics = lyrics[lyrics['language']=='en']

# #Only keep popular artists, with genre Rock/Pop and popularity high enough
# artists = pd.read_csv('artists-data.csv')
# artists = artists[(artists['Genres'].isin(['Rock'])) & (artists['Popularity']>5)]
df = lyrics
# df = df.drop(columns=['ALink','SLink','language','Link'])

# #Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
df = df[df['text'].apply(lambda x: len(x.split(' ')) < 350)]

# #Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

# #Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

# #For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_text'] = test_set['text'].str.split().str[-20:].apply(' '.join)
test_set['text'] = test_set['text'].str.split().str[-20:].apply(' '.join)

In [3]:
print(test_set["text"][0])

Предпоследний цветок колокольчика медленно погрузился в зелье.


# Загрузка модели и Токенизатора

In [4]:
class SongLyrics(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['text']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = SongLyrics(df['text'], truncate=True, gpt2_type="gpt2")  

Token indices sequence length is longer than the specified maximum sequence length for this model (1219 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
# Для наглядности будем работать с русскоязычной GPT от Сбера.
# Ниже команды для загрузки и инициализации модели и токенизатора.
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

# Функция обучения модели

In [6]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

# Модель генерации данных

In [7]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                generated = generated.to("cuda")
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("."):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.to("cpu").squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.to("cpu").squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}." 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cuda'), tokenizer, test_data['text'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

#Run the functions to generate the lyrics
generated_lyrics = text_generation(test_set)

100%|██████████| 1/1 [00:00<00:00,  1.90it/s]


In [8]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['text'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final

In [9]:
test_set

,index,Unnamed: 0,lable,text,True_end_text,Generated_lyrics
0,21170,21171,1,Предпоследний цветок колокольчика медленно пог...,Предпоследний цветок колокольчика медленно пог...,"\n\n– Итак, кто первый, Митра?\n\n– Чего ты хо..."
1,15222,15222,1,"профессор Зельеварения на самом деле, но, к со...","профессор Зельеварения на самом деле, но, к со...","\n\nОна была уверена, что её внешность была об..."
2,11718,11718,1,— Ничуточки. Не будь я тебе обязана...,— Ничуточки. Не будь я тебе обязана...,\n\nНа ночь в ночь он прихватил с собой гитару...
3,11227,11227,1,"— Спасибо, Белла, — ответил бесстрастный голос...","— Спасибо, Белла, — ответил бесстрастный голос...","— Он осторожно высвободил руку, которой, каза..."
4,22079,22080,1,"Хогвартса, с нашим профессором Квирреллом. Это...","Хогвартса, с нашим профессором Квирреллом. Это...","\n\nВспомнив это, Маклагген покачал головой."
...,...,...,...,...,...,...
195,21418,21419,1,"но не забывай об этом, ― в его руке снова оказ...","но не забывай об этом, ― в его руке снова оказ...",\n\nДжеки подчинился.
196,7229,7229,1,— Что за бре...,— Что за бре...,"— Так начиналась фраза, но вовремя оборвалась..."
197,20916,20917,1,что я говорил тебе раньше про заклинание крест...,что я говорил тебе раньше про заклинание крест...,Но не делай глупостей.
198,1737,1737,1,этот провал обозначает потерю всего. Ты веришь...,этот провал обозначает потерю всего. Ты веришь...,"Вместо того, чтобы признать правду и признать..."


In [10]:
for i in range(5):
    print(test_set['Generated_lyrics'][i])



– Итак, кто первый, Митра?

– Чего ты хочешь, Великий.


Она была уверена, что её внешность была обманчивой.


На ночь в ночь он прихватил с собой гитару, а днем, когда солнце только-только начало припекать, вдруг совершил небольшой.
 — Он осторожно высвободил руку, которой, казалось, касался ее подбородок.


Вспомнив это, Маклагген покачал головой.
